Imports
-------

In [1]:
from typing import List, Tuple, Union
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import sklearn.model_selection as sk
from tensorflow.keras.utils import to_categorical

Data generation
---------------

In [46]:
def get_gaussian(amplitude, x, center, width):
    return amplitude * np.exp(-((x - center) ** 2) / (2 * width ** 2))

def generate_gaussian_data(
    num_samples, 
    input_length, 
    num_gaussian=(1, 5), 
    amplitude_range=(1, 5), 
    center_range=(32, 96), 
    width_range=(5, 20), 
    noise_amplitude=0.0):
    """
    Generate a dataset of Gaussian curves, with a variable or fixed number of peaks per slice,
    and optional Gaussian noise.

    Parameters
    ----------
    num_samples : int
        Number of slices to generate.
    input_length : int
        Length of each slice.
    num_gaussian : int or tuple of int
        If int, fixed number of Gaussian peaks per slice.
        If tuple, (min_peaks, max_peaks), a random number of peaks per slice within this range.
    amplitude_range : tuple of float
        Range of amplitudes for the Gaussian peaks.
    center_range : tuple of int
        Range of center positions for the Gaussian peaks.
    width_range : tuple of float
        Range of standard deviations (widths) for the Gaussian peaks.
    noise_amplitude : float
        Standard deviation of the Gaussian noise added to each slice.

    Returns
    -------
    X_train : numpy.ndarray
        Array of slices with Gaussian peaks and added noise, shape (num_samples, input_length, 1).
    y_train : numpy.ndarray
        Array of maximum values for each peak in every slice, shape (num_samples, max_peaks).
    num_peaks : numpy.ndarray
        Array with the number of peaks in each slice, shape (num_samples,).
    peak_positions : numpy.ndarray
        Array with the positions of peaks in each slice, shape (num_samples, max_peaks).
        Unused entries are filled with zeros.
    """
    if isinstance(num_gaussian, tuple):
        min_peaks, max_peaks = num_gaussian
    else:
        min_peaks = max_peaks = num_gaussian

    max_peaks = max(max_peaks, 1)  # Ensure at least one peak
    X_train = np.zeros((num_samples, input_length, 1))
    amplitudes_array = np.zeros((num_samples, max_peaks))  # Store amplitudes of peaks
    num_peaks_array = np.zeros(num_samples, dtype=int)  # Store the number of peaks per slice
    peak_positions_array = np.zeros((num_samples, max_peaks))  # Store the positions of peaks
    peak_widths_array = np.zeros((num_samples, max_peaks))
        
    x = np.linspace(0, input_length - 1, input_length)

    for i in range(num_samples):
        slice_curve = np.zeros(input_length)  # Initialize the slice

        # Determine the number of peaks for this slice
        num_peaks_array[i] = np.random.randint(min_peaks, max_peaks + 1)
        peak_amplitudes = []
        peak_centers = []
        peak_widths = []

        for _ in range(num_peaks_array[i]):
            amplitude = np.random.uniform(*amplitude_range)
            center = np.random.uniform(*center_range)
            width = np.random.uniform(*width_range)

            # Gaussian curve: y = A * exp(-((x - center)^2) / (2 * width^2))
            slice_curve += get_gaussian(amplitude, x, center, width)  # Add the Gaussian peak to the slice
            peak_amplitudes.append(amplitude)
            peak_centers.append(center)
            peak_widths.append(width)

        # Add Gaussian noise to the slice
        noise = np.random.normal(0, noise_amplitude, input_length)
        slice_curve += noise

        # Update the arrays
        X_train[i, :, 0] = slice_curve
        amplitudes_array[i, :num_peaks_array[i]] = sorted(peak_amplitudes, reverse=True)
        peak_positions_array[i, :num_peaks_array[i]] = peak_centers
        peak_widths_array[i, :num_peaks_array[i]] = peak_widths

    return X_train, amplitudes_array, num_peaks_array, peak_positions_array, peak_widths_array


Model generation
----------------

In [47]:
def build_model(input_length, max_number_of_peaks):
    """
    Build a model that predicts both the maximum value and the position of the maximum value
    from the input signal.

    Parameters
    ----------
    input_length : int
        Length of the input signal (e.g., 128).

    Returns
    -------
    tensorflow.keras.Model
        The constructed Keras model.
    """
    input_layer = layers.Input(shape=(input_length, 1))

    # Feature extraction
    x = layers.Conv1D(filters=32, kernel_size=3, activation="relu", padding="same")(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(filters=64, kernel_size=3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Attention mechanism
    # attention = layers.Attention()([x, x])
    # query = layers.Dense(64)(x)  # Learnable query
    # key = layers.Dense(64)(x)    # Keys
    # value = layers.Dense(64)(x)  # Values
    attention = layers.Attention()([x, x])

    x = layers.GlobalAveragePooling1D()(attention)
    

    # Flatten and dense layers
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)

    # Output layer
    num_peaks_output = layers.Dense(max_number_of_peaks + 1, activation="softmax", name="num_peaks")(x)

    return models.Model(inputs=input_layer, outputs=num_peaks_output)

Utils
-----

In [176]:
from itertools import islice

def batched(iterable, n):
    # batched('ABCDEFG', 3) → ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        yield batch

        
def dataset_split(test_size, random_state, max_number_of_peaks, **kwargs):

    values = list(kwargs.values())

    splitted = sk.train_test_split(*values, test_size=0.2, random_state=42)

    output = {
        'train': dict(), 'test': dict()
    }
    
    for (k, v), (train_data, test_data) in zip(kwargs.items(), batched(splitted, 2)):
        if k == 'num_peaks':
            train_data =  to_categorical(train_data, max_number_of_peaks + 1)
            test_data =  to_categorical(test_data, max_number_of_peaks + 1)
        
        output['train'][k] = train_data
        output['test'][k] = test_data

    return output


def plot_model_performance(history, model, validation_data, num_examples=5):
    """
    Plot training history and visualize validation cases for a trained model.

    Parameters
    ----------
    history : tensorflow.keras.callbacks.History
        The training history object from model.fit().
    model : tensorflow.keras.Model
        The trained Keras model.
    validation_data : tuple
        Tuple containing validation inputs and expected outputs:
        (X_val, y_val).
    num_examples : int, optional
        Number of validation cases to visualize. Default is 5.
    """
    plt.close('all')
    # Visualize validation cases in subplots
    num_rows = (num_examples + 1)

    fig, axes = plt.subplots(num_rows, 1, figsize=(12, num_rows * 3))

    ax = axes[0]
    twin_ax = ax.twinx()
    ax.plot(history.history['loss'], label='Training Loss', color='C0')
    twin_ax.plot(history.history['val_accuracy'], color='C1', label='Validation Accuracy')
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.legend()
    twin_ax.legend()

    indices = np.random.choice(len(validation_data['test']['raw']), num_examples, replace=False)
    for ax, (idx, _) in zip(axes[1:], enumerate(indices)):
        true_num_peaks = validation_data['test']['num_peaks'][idx]
        true_peak_positions = validation_data['test']['positions'][idx]
        true_peak_widths = validation_data['test']['widths'][idx]
        true_peak_amplitudes = validation_data['test']['amplitudes'][idx]
        
        x = np.arange(128)
        
        input_signal = validation_data['test']['raw'][idx, :, 0]
        
        
        predicted_num_peaks = model.predict(validation_data['test']['raw'][idx:idx + 1])[0]
        
        ax.plot(input_signal, label=f"GD: {np.argmax(true_num_peaks)} -- Pred: {np.argmax(predicted_num_peaks)}")

        for position, amplitude, width in zip(true_peak_positions, true_peak_amplitudes, true_peak_widths):
            if position !=0:
                ax.plot(x, get_gaussian(amplitude, x, position, width), linestyle='--', color='black')
                ax.axvline(position, color='red')

        ax.legend()

    plt.tight_layout()
    plt.show()

Script
------

In [178]:
# Generate data with up to 5 peaks per slice
INPUT_LENGTH = 128
MAX_NUMBER_OF_PEAKS = 4

gaussian_slices, amplitudes, num_peaks, positions, widths = generate_gaussian_data(
    num_samples=400,
    input_length=INPUT_LENGTH,
    num_gaussian=(1, max_number_of_peaks),
    amplitude_range=(2, 5),
    center_range=(20, 100),
    width_range=(5, 5),
    noise_amplitude=0.2
)

data = dataset_split(
    raw=gaussian_slices, 
    positions=positions, 
    num_peaks=num_peaks, 
    amplitudes=amplitudes, 
    widths=widths, 
    test_size=0.2, 
    random_state=42, 
    max_number_of_peaks=MAX_NUMBER_OF_PEAKS
)


model = build_model(input_length=INPUT_LENGTH, max_number_of_peaks=MAX_NUMBER_OF_PEAKS)


model.compile(
    optimizer='adam',
    loss={'num_peaks': 'categorical_crossentropy',},
    metrics={'num_peaks': ['accuracy'],}
)

history = model.fit(
    data['train']['raw'],
    data['train']['num_peaks'],
    validation_data=(data['test']['raw'], data['test']['num_peaks']),
    epochs=120,
    batch_size=32
)

%matplotlib qt
validation_data = (slices_validation, num_peaks_validation, amplitudes_validation, positions_validation, widths_validation)

plot_model_performance(
    history=history, 
    model=model, 
    validation_data=data, 
    num_examples=5
)

Epoch 1/120
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3502 - loss: 1.5175 - val_accuracy: 0.2000 - val_loss: 1.5332
Epoch 2/120
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6598 - loss: 1.0331 - val_accuracy: 0.2000 - val_loss: 1.4680
Epoch 3/120
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7252 - loss: 0.7105 - val_accuracy: 0.2000 - val_loss: 1.4226
Epoch 4/120
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7936 - loss: 0.5572 - val_accuracy: 0.2500 - val_loss: 1.3994
Epoch 5/120
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7541 - loss: 0.5209 - val_accuracy: 0.3000 - val_loss: 1.3584
Epoch 6/120
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7933 - loss: 0.4544 - val_accuracy: 0.3750 - val_loss: 1.3107
Epoch 7/120
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8367 - loss: 0.4183 - val_accuracy: 0.4125 - val_loss: 1.2357
Epoch 8/120
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8186 - loss: 0.4189 - val_accuracy: 0.4